In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import exposure

In [6]:
def shadow_remove(img):
    rgb_planes = cv2.split(img)
    result_norm_planes = []
    for plane in rgb_planes:
        dilated_img = cv2.dilate(plane, np.ones((7,7), np.uint8))
        bg_img = cv2.medianBlur(dilated_img, 21)
        diff_img = 255 - cv2.absdiff(plane, bg_img)
        norm_img = cv2.normalize(diff_img,None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
        result_norm_planes.append(norm_img)
    shadowremov = cv2.merge(result_norm_planes)
    return shadowremov

In [7]:
# def hog_preprocessing(image):
#     image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     resized_image = cv2.resize(image,(500,500))
#     shad_image = shadow_remove(resized_image)
#     grayscale_image = cv2.cvtColor(shad_image, cv2.COLOR_BGR2GRAY)
#     filtered_image  = cv2.GaussianBlur(grayscale_image, (3, 3), 1.0)
#     clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
#     normalized_image = clahe.apply(filtered_image)
#     img_array = np.array(normalized_image, dtype=np.float32) / 255.0
#     corrected_array = np.power(img_array, 0.5)
#     corrected_image = np.uint8(corrected_array * 255.0)
#     resized_image = cv2.resize(corrected_image,(128,128))
#     return resized_image

In [8]:
def hog_preprocessing(image):
    image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    filtered_image  = cv2.GaussianBlur(image, (3, 3), 1.0)
    # resized_image = cv2.resize(filtered_image,(500,500))
    # scale_percent = 10
    # # percent of original size
    # width = int(image.shape[1] * scale_percent / 100)
    # height = int(image.shape[0] * scale_percent / 100)
    dim = (500, 500)
    resized_image = cv2.resize(filtered_image,dim)
    #======================================Color Segmentation========================#
    hsv_image = cv2.cvtColor(resized_image, cv2.COLOR_RGB2HSV)
    light_skin = (0, 50, 50 )
    dark_skin = (50, 250, 255)
    mask = cv2.inRange(hsv_image, light_skin, dark_skin)
    #Define the kernel for closing
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    # Perform closing on the thresholded image)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel,iterations=10)
    result = cv2.bitwise_and(resized_image, resized_image, mask=mask)
    # resized_image = cv2.resize(result,(128,128))
    # scale_percent = 30
    # # percent of original size
    # width = int(image.shape[1] * scale_percent / 100)
    # height = int(image.shape[0] * scale_percent / 100)
    dim = (128, 128)
    resized_image = cv2.resize(result,dim)
    grayscale_image = cv2.cvtColor(resized_image, cv2.COLOR_RGB2GRAY)
    #=====================================Contrast enhancement========================#
    # Apply histogram equalization to enhance contrast
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(10,10))
    normalized_image = clahe.apply(grayscale_image)
    gamma_corrected = exposure.adjust_gamma(normalized_image, gamma=2.5)
    return gamma_corrected